In [1]:
!pip install openai

In [2]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model = 'gpt-5-nano',
    messages = [
        {'role': 'system', 'content': 'You are helpful assistant.'},
        {'role': 'user',   'content': '만나서 반가워요!'}
    ]
)

print(response.to_json(indent = 2))

In [ ]:
response = client.chat.completions.create(
    model = 'gpt-5-nano',
    messages = [
        {'role': 'system',    'content': 'You are helpful assistant.'},
        {'role': 'user',      'content': '만나서 반가워요!'},
        {'role': 'assistant', 'content': '저도 만나서 반가워요! 무엇을 도와드릴까요? 궁금한 점이 있으면 물어보시고, 대화 주제나 하고 싶은 작업을 말해주시면 함께 진행할게요.'},
        {'role': 'user',      'content': 'RAG를 주제로 대화해보자.'},
    ],
)

print(response.to_json(indent = 2))

({'role': 'user', 'content': 'RAG를 주제로 대화해보자.'},)

In [ ]:
response = client.chat.completions.create(
    model = 'gpt-5-nano',
    messages = [
        {'role': 'system', 'content': '당신은 유용한 assistant입니다. 제 질문에 간략하게 응답해 주세요.'},
        {'role': 'user',   'content': '안녕하세요! 저는 Edward라고 합니다.'},
    ],
    stream = True,
)

# chunk 구조
# ChatCompletionChunk(id='chatcmpl-C67Z9bzqiG7VmhWw7Ni2xszlOP6YI', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1755575087, model='gpt-5-nano-2025-08-07', object='chat.completion.chunk', service_tier='default', system_fingerprint=None, usage=None, obfuscation='iK')
for chunk in response:
  content = chunk.choices[0].delta.content
  if (content is not None):
    print(content, end = "", flush = True)

반갑습니다, Edward님. 무엇을 도와드릴까요?

In [ ]:
# 1. 사용 가능한 함수 목록과 질문 → LLM
# 2. LLM이 함수 사용을 원한다는 응답 → Python 프로그램에서 해당 함수 실행
# 3. 실행 결과 → LLM → 최종 답변
import json

def get_current_weather(location, unit="fahrenheit"):
  if "seoul" in location.lower():
    return json.dumps({"location": "Seoul", "temperature": "10", "unit": "unit"})
  elif "san francisco" in location.lower():
    return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "unit"})
  elif "paris" in location.lower():
    return json.dumps({"location": "Paris", "temperature": "22", "unit": "unit"})
  else:
    return json.dumps({"location": "Location", "temperature": "unknown"})

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"]
                    },
                },
                "required": ["location"],
            },
        },
    }
]

from openai import OpenAI

client = OpenAI()

messages = [
    {
        "role": "user",
        "content": "서울 날씨는 어떤가요?",
    }
]

response = client.chat.completions.create(
    model = 'gpt-5-nano',
    messages = messages,
    tools = tools,
)

print(response.to_json(indent = 2))

response_message = response.choices[0].message

messages.append(response_message.to_dict())

{
  "id": "chatcmpl-C8Cx65AUd5MT6SI9MGnOOqpzJqq95",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "tool_calls": [
          {
            "id": "call_SwntwNIatBrcuqJ4DmDD8ZNR",
            "function": {
              "arguments": "{\"location\":\"Seoul, South Korea\",\"unit\":\"celsius\"}",
              "name": "get_current_weather"
            },
            "type": "function"
          }
        ]
      }
    }
  ],
  "created": 1756072448,
  "model": "gpt-5-nano-2025-08-07",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 289,
    "prompt_tokens": 161,
    "total_tokens": 450,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 256,
      "rejected_prediction_tokens": 0
 

In [ ]:
available_functions = {
    "get_current_weather": get_current_weather
}

# 사용하고 싶은 함수가 여러 개일 수 있으므로 반복문 사용
for tool_call in response_message.tool_calls:
  function_name = tool_call.function.name
  function_to_call = available_functions[function_name]
  function_args = json.loads(tool_call.function.arguments)
  function_response = function_to_call(
      location = function_args.get("location"),
      unit = function_args.get("unit")
  )
  print(function_response)

# 함수 실행 결과를 대화 이력으로 messages에 추가
messages.append(
    {
        "tool_call_id": tool_call.id,
        "role": "tool",
        "name": function_name,
        "content": function_response
    }
)

for i in messages:
  print(i)

{"location": "Seoul", "temperature": "10", "unit": "unit"}
{'role': 'user', 'content': '서울 날씨는 어떤가요?'}
{'content': None, 'refusal': None, 'role': 'assistant', 'annotations': [], 'tool_calls': [{'id': 'call_SwntwNIatBrcuqJ4DmDD8ZNR', 'function': {'arguments': '{"location":"Seoul, South Korea","unit":"celsius"}', 'name': 'get_current_weather'}, 'type': 'function'}]}
{'tool_call_id': 'call_SwntwNIatBrcuqJ4DmDD8ZNR', 'role': 'tool', 'name': 'get_current_weather', 'content': '{"location": "Seoul", "temperature": "10", "unit": "unit"}'}


In [ ]:
print(json.dumps(messages, ensure_ascii=False, indent=2))

[
  {
    "role": "user",
    "content": "서울 날씨는 어떤가요?"
  },
  {
    "content": null,
    "refusal": null,
    "role": "assistant",
    "annotations": [],
    "tool_calls": [
      {
        "id": "call_SwntwNIatBrcuqJ4DmDD8ZNR",
        "function": {
          "arguments": "{\"location\":\"Seoul, South Korea\",\"unit\":\"celsius\"}",
          "name": "get_current_weather"
        },
        "type": "function"
      }
    ]
  },
  {
    "tool_call_id": "call_SwntwNIatBrcuqJ4DmDD8ZNR",
    "role": "tool",
    "name": "get_current_weather",
    "content": "{\"location\": \"Seoul\", \"temperature\": \"10\", \"unit\": \"unit\"}"
  }
]


In [ ]:
second_response = client.chat.completions.create(
    model = 'gpt-5-nano',
    messages=messages,
)

print(second_response.to_json(indent = 2))

{
  "id": "chatcmpl-C8DJIY1AbCQtdm7clOEJIYWA1hWrw",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "현재 서울의 기온은 섭씨 10도예요.\n\n다음 정보가 필요하시면 알려 주세요: 바람/습도, 강수 확률, 날씨 상태(맑음/구름 많음 등) 등. 옷차림 추천으로는 겉옷 얇은 자켓 정도 준비하시면 좋습니다.",
        "refusal": null,
        "role": "assistant",
        "annotations": []
      }
    }
  ],
  "created": 1756073824,
  "model": "gpt-5-nano-2025-08-07",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 788,
    "prompt_tokens": 72,
    "total_tokens": 860,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 704,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# 브라우저 실행
driver = webdriver.Chrome(ChromeDriverManager().install())

try:
    # 네이버 접속
    driver.get('https://www.naver.com')

    # 검색창에 텍스트 입력
    search_box = driver.find_element(By.ID, 'query')
    search_box.send_keys('파이썬 크롤링')

    # 검색 버튼 클릭
    search_button = driver.find_element(By.ID, 'search_btn')
    search_button.click()

    # 검색 결과 대기
    wait = WebDriverWait(driver, 10)
    results = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.result')))

    # 검색 결과 출력
    for result in results[:5]:  # 상위 5개만
        title = result.find_element(By.CSS_SELECTOR, '.title').text
        print(title)

finally:
    driver.quit()

ModuleNotFoundError: No module named 'selenium'